In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
!pip install fuzzywuzzy
from fuzzywuzzy import process

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
ratings = pd.read_csv('ratings_small.csv')
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
movie_names = pd.read_csv('movies_metadata.csv').drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).drop([19730, 29503, 35587])

Mounted at /content/drive


<ipython-input-2-333e1ac78e24>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_names = pd.read_csv('/content/drive/MyDrive/movies_metadata.csv').drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).drop([19730, 29503, 35587])


In [ ]:
ratings = ratings.drop('timestamp', axis = 1)
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [ ]:
movie_names = movie_names.rename(columns={'original_title': 'title'})
movie_names = movie_names[['title', 'genres']]
movie_names.head()

,title,genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"


In [ ]:
movie_data = pd.concat([ratings, movie_names], axis=1)
movie_data[movie_data['rating'] > 4].head(30)

,userId,movieId,rating,title,genres
21,2,17,5.0,Copycat,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name..."
22,2,39,5.0,Assassins,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
29,2,150,5.0,摇啊摇，摇到外婆桥,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name..."
37,2,222,5.0,It Takes Two,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '..."
44,2,265,5.0,To Die For,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na..."
45,2,266,5.0,How To Make An American Quilt,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n..."
83,2,551,5.0,Last Summer in the Hamptons,[]
85,2,585,5.0,White Squall,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
89,2,589,5.0,The Journey of August King,"[{'id': 18, 'name': 'Drama'}]"
90,2,590,5.0,Mary Reilly,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name..."


In [ ]:
pattern = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
pattern['total number of ratings'] = pd.DataFrame(movie_data.groupby('title')['rating'].count()) 
pattern.head()

,rating,total number of ratings
title,,
!Women Art Revolution,4.5,1
#1 Cheerleader Camp,5.0,1
#Horror,2.5,1
#Pellichoopulu,4.0,1
#SELFIEPARTY,4.5,1


In [ ]:
pivot = ratings.pivot(index=['userId'], columns=['movieId'], values='rating').fillna(0)
pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from scipy.sparse import csr_matrix
mat=csr_matrix(pivot.values)
mat

<671x9066 sparse matrix of type '<class 'numpy.float64'>'
	with 100004 stored elements in Compressed Sparse Row format>

In [ ]:
knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [ ]:
knn.fit(mat)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [ ]:
def KNN_Movie_Recommender(mn, data=mat, model=knn, n_recommendations=21):
    # model.fit(data)
    i = process.extractOne(mn, movie_names['title'])[2]
    d, i = model.kneighbors(data[i], n_neighbors=n_recommendations)
    rmi = sorted(list(zip(i.squeeze().tolist(),d.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
    l = []
    for val in rmi:
        l.append({'Title':movie_names['title'][val[0]],'Distance':val[1]})
    res = pd.DataFrame(l, index = range(1,n_recommendations))
    return res

In [ ]:
results = KNN_Movie_Recommender('Toy Story')

In [ ]:
movie_info = results[['Title']].values.tolist()

In [ ]:
movie_info

[['The Ref'],
 ['Naked'],
 ["I'll Do Anything"],
 ['Desert Winds'],
 ['The Celluloid Closet'],
 ['So I Married an Axe Murderer'],
 ["Jason's Lyric"],
 ['Race the Sun'],
 ['In the Mouth of Madness'],
 ['Unforgettable'],
 ['飲食男女'],
 ['Frankie Starlight'],
 ['Made in America'],
 ['The Stars Fell on Henrietta'],
 ['Carrington'],
 ["White Man's Burden"],
 ['The Swan Princess'],
 ['The Adventures of Priscilla, Queen of the Desert'],
 ['Roula - Dunkle Geheimnisse'],
 ['Star Trek: Generations']]